<a href="https://colab.research.google.com/github/giacomosansoni/Siamese_Network_Artist_Similarity/blob/main/Embeddings_Posts_Captions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Install

In [ ]:
!pip install git+https://github.com/openai/CLIP.git
#!pip install clip

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-_1fv4ss6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-_1fv4ss6
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 756.9 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [ ]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 8.1 MB/s eta 0:00:00


In [ ]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 10.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=0cb3e8ccecd7572d2482c9a1adad42fc2edec4cc0965eeaea22befe4d92b4edf
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

###1

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Tesi/Thesis/
#drive.mount('/content/drive')
#%cd "/content/drive/MyDrive/Università/tesi-artist-similarity/giacomo/Thesis"

Mounted at /gdrive
/gdrive/.shortcut-targets-by-id/1D4FFhBTYn6QfBuxB5OXjn7JgI3z14YxH/Thesis


In [ ]:
import os
import clip
import torch
import pandas as pd
from PIL import Image
from transformers import AutoTokenizer
import numpy as np
import re
import emoji
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from tqdm import tqdm
from googletrans import Translator
translator = Translator()

def preprocess_caption1(caption):
  # Removes emji
  clean_text = emoji.replace_emoji(caption, replace='<EMOJI>')
  # Remove user mentions
  caption = re.sub(r'@\w+', '@user', clean_text)  # Removes @mentions
  caption = translator.translate(caption).text
  return caption.strip()

def preprocess_caption2(caption):
  # Removes emji
  clean_text = emoji.replace_emoji(caption, replace='<EMOJI>')
  # Remove user mentions
  caption = re.sub(r'@\w+', '@user', clean_text)  # Removes @mentions
  return caption.strip()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from sentence_transformers import SentenceTransformer, util

root_dir = os.getcwd()
#artists_dir = os.path.join(root_dir, 'Posts2')
artists_dir = os.path.join(root_dir, 'instagram')
emb_dir = os.path.join(root_dir, 'Embeddings')

ig_progress = pd.read_csv('instagram_progress_mio.csv')
artists = pd.read_csv('artists_mio.csv')

model, preprocess = clip.load('ViT-L/14@336px', device=device) # Images model
model1 = SentenceTransformer('all-distilroberta-v1').to(device) # MAX LENGTH = 256
model2 = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2').to(device) # MAX LENGTH = 128

tokenizer1 = AutoTokenizer.from_pretrained('sentence-transformers/all-distilroberta-v1')
tokenizer2 = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

100%|███████████████████████████████████████| 891M/891M [00:50<00:00, 18.5MiB/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def img_emb(file_path):
    image = Image.open(file_path)
    image_input = preprocess(image).unsqueeze(0).to(device)  # Move input tensor to the correct device
    with torch.no_grad():
      image_features = model.encode_image(image_input)
      # Squeeze the batch dimension out if it's size 1 (usually index 0)
      image_features = image_features.squeeze(0)
    return image_features

def txt_emb(file_path):
    with open(file_path, 'r', encoding='utf-8') as file: # with open(file_path, 'r', encoding='utf-8') as file: ?  or ISO-8859-1
      caption = file.readlines()
    caption1 = preprocess_caption1(caption)
    caption2 = preprocess_caption2(caption)
    num_tokens1 = len(tokenizer1.tokenize(caption1))
    num_tokens2 = len(tokenizer2.tokenize(caption2))

    if num_tokens1 > 256:
      sentences1 = sent_tokenize(' '.join(caption1))
      num_sentences1 = len(sentences1)
      sentences1_1 = sentences1[0:int(num_sentences1/2)]
      sentences1_1 = combined_sentence = ' '.join(sentences1_1)
      sentences1_2 = sentences1[int(num_sentences1/2):]
      sentences1_2 = ' '.join(sentences1_2)
      with torch.no_grad():
        emb1 = model1.encode(sentences1_1, convert_to_tensor=True).to(device)
        emb2 = model1.encode(sentences1_2, convert_to_tensor=True).to(device)
      embedding1_1 = (emb1+emb2)/2
    else:
      with torch.no_grad():
        embedding1_1 = model1.encode(caption1, convert_to_tensor=True).to(device)

    if num_tokens2 > 128:
      sentences2 = sent_tokenize(' '.join(caption2))
      num_sentences2 = len(sentences2)
      sentences2_1 = sentences2[0:int(num_sentences2/2)]
      sentences2_1 = ' '.join(sentences2_1)
      sentences2_2 = sentences2[int(num_sentences2/2):]
      sentences2_2 = ' '.join(sentences2_2)
      with torch.no_grad():
        emb1 = model2.encode(sentences2_1, convert_to_tensor=True).to(device)
        emb2 = model2.encode(sentences2_2, convert_to_tensor=True).to(device)
      embedding2_1 = (emb1+emb2)/2
    else:
      with torch.no_grad():
        embedding2_1 = model2.encode(caption2, convert_to_tensor=True).to(device)

    full_caption1 = ' '.join(caption1)
    full_caption2 = ' '.join(caption2)
    # Tokenize the full caption text into sentences
    sentences1 = sent_tokenize(full_caption1)
    sentences2 = sent_tokenize(full_caption2)
    with torch.no_grad():
      embedding1_2 = model1.encode(sentences1, convert_to_tensor=True).to(device)
      embedding2_2 = model2.encode(sentences2, convert_to_tensor=True).to(device)
    embedding1_2 = torch.mean(embedding1_2, dim=0)
    embedding2_2 = torch.mean(embedding2_2, dim=0)

    return embedding1_1, embedding1_2, embedding2_1, embedding2_2

def process_artist(directory):
    print("PROCESS FUNCTION")
    file_dict = {}
    ids = []
    img_embeddings = []
    caption_emb1_1 = []
    caption_emb1_2 = []
    caption_emb2_1 = []
    caption_emb2_2 = []

    for file_name in os.listdir(directory):
      if file_name.endswith('.txt'):
        base_id = re.match(r"(.+)\.txt$", file_name).group(1)
        file_dict[base_id] = {'jpg': [], 'txt': file_name}

        # Step 2: Match jpg files to their corresponding txt files based on full base_id match
    for file_name in os.listdir(directory):
      if file_name.endswith('.jpg'):
        # Here we check if the jpg file starts with any of the established base IDs
        for base_id in file_dict:
          pattern = re.escape(base_id) + r"(_\d+)?\.jpg$"
          if re.match(pattern, file_name):
            file_dict[base_id]['jpg'].append(file_name)
            break

    # Process files according to their types
    for base_id, files in file_dict.items():
        jpg_results = []
        txt_result = None

        number_of_jpg_files = len(files['jpg'])
        if (number_of_jpg_files > 1):
          print(f"{base_id} has {number_of_jpg_files} images!")

        # Process .jpn files
        for jpg_file in files['jpg']:
            full_path = os.path.join(directory, jpg_file)
            jpg_results.append(img_emb(full_path))

        # Process .txt file
        if files['txt']:
            full_path = os.path.join(directory, files['txt'])
            txt_emb1_1, txt_emb1_2, txt_emb2_1, txt_emb2_2 = txt_emb(full_path)
        else:
          print(f"No caption for post {base_id} of artist {directory}")

        # Store results
        ids.append(base_id)
        if jpg_results:
            img_embeddings.append(torch.mean(torch.stack(jpg_results), dim=0))
        if txt_emb1_1 is not None:
            caption_emb1_1.append(txt_emb1_1)
        else:
            print(f"text emb1_1 has problems")
        if txt_emb1_2 is not None:
            caption_emb1_2.append(txt_emb1_2)
        else:
            print(f"text emb1_2 has problems")
        if txt_emb2_1 is not None:
            caption_emb2_1.append(txt_emb2_1)
        else:
            print(f"text emb2_1 has problems")
        if txt_emb2_2 is not None:
            caption_emb2_2.append(txt_emb2_2)
        else:
            print(f"text emb2_2 has problems")
    print("Everything ok")
    return ids, img_embeddings, caption_emb1_1, caption_emb1_2, caption_emb2_1, caption_emb2_2

In [ ]:
# NO 40
'''for folder_name in tqdm(os.listdir(artists_dir)):
    print(folder_name)
    instagram_link = artists.loc[artists['musicbrainz_id'] == folder_name, 'instagram_spotify'].values[0]
    username = instagram_link.split('/')[-2]

    file_name = f"{folder_name}.npz"
    npz_file_path = os.path.join(root_dir, 'Posts2', file_name)
    folder_path = os.path.join(artists_dir, folder_name)
    if not os.path.isdir(folder_path):
      continue
    if os.path.exists(npz_file_path):
      print(f"{folder_name} already done")
      continue
    if os.path.isdir(folder_path):
      ids, img_embeddings, embeddings1_1, embeddings1_2, embeddings2_1, embeddings2_2 = process_artist(folder_path)


    if ids and embeddings1_1 and embeddings1_2 and embeddings2_1 and embeddings2_2 and img_embeddings:
      # Convert lists to numpy arrays
      ids_array = np.array(ids)
      embeddings1_1_array = np.stack([emb.cpu() for emb in embeddings1_1])  # Move tensors to CPU
      embeddings1_2_array = np.stack([emb.cpu() for emb in embeddings1_2])  # Move tensors to CPU
      embeddings2_1_array = np.stack([emb.cpu() for emb in embeddings2_1])  # Move tensors to CPU
      embeddings2_2_array = np.stack([emb.cpu() for emb in embeddings2_2])
      img_embeddings_array = np.stack([emb.cpu() for emb in img_embeddings])

      # Save the arrays into an .npz file
      np.savez(npz_file_path, ids=ids_array, img_embeddings = img_embeddings_array, embeddings1_1=embeddings1_1_array, embeddings1_2=embeddings1_2_array, embeddings2_1=embeddings2_1_array, embeddings2_2=embeddings2_2_array)

    else:
      print(f"No valid .txt files or embeddings generated for folder: {folder_name}")'''

'for folder_name in tqdm(os.listdir(artists_dir)):\n    print(folder_name)\n    instagram_link = artists.loc[artists[\'musicbrainz_id\'] == folder_name, \'instagram_spotify\'].values[0]\n    username = instagram_link.split(\'/\')[-2]\n\n    file_name = f"{folder_name}.npz"\n    npz_file_path = os.path.join(root_dir, \'Posts2\', file_name)\n    folder_path = os.path.join(artists_dir, folder_name)\n    if not os.path.isdir(folder_path):\n      continue\n    if os.path.exists(npz_file_path):\n      print(f"{folder_name} already done")\n      continue\n    if os.path.isdir(folder_path):\n      ids, img_embeddings, embeddings1_1, embeddings1_2, embeddings2_1, embeddings2_2 = process_artist(folder_path)\n\n\n    if ids and embeddings1_1 and embeddings1_2 and embeddings2_1 and embeddings2_2 and img_embeddings:\n      # Convert lists to numpy arrays\n      ids_array = np.array(ids)\n      embeddings1_1_array = np.stack([emb.cpu() for emb in embeddings1_1])  # Move tensors to CPU\n      embeddi

In [ ]:
# SI 40
for folder_name in tqdm(os.listdir(artists_dir)):
    folder_path = os.path.join(artists_dir, folder_name)
    if not os.path.isdir(folder_path):
      continue
    print(folder_name)
    instagram_link = artists.loc[artists['musicbrainz_id'] == folder_name, 'instagram_spotify'].values[0]
    username = instagram_link.split('/')[-2]

    file_name = f"{folder_name}.npz"
    npz_file_path = os.path.join(root_dir, 'Embeddings', file_name)
    npz_file_path2 = os.path.join(root_dir, 'Embeddings_40', file_name)
    if not os.path.isdir(folder_path):
      continue
    if os.path.exists(npz_file_path):
      print(f"{folder_name} already done")
      continue
    if os.path.isdir(folder_path):
      ids, img_embeddings, embeddings1_1, embeddings1_2, embeddings2_1, embeddings2_2 = process_artist(folder_path)

    if ids and embeddings1_1 and embeddings1_2 and embeddings2_1 and embeddings2_2 and img_embeddings:
      # Convert lists to numpy arrays
      ids_array = np.array(ids)
      embeddings1_1_array = np.stack([emb.cpu() for emb in embeddings1_1])  # Move tensors to CPU
      embeddings1_2_array = np.stack([emb.cpu() for emb in embeddings1_2])  # Move tensors to CPU
      embeddings2_1_array = np.stack([emb.cpu() for emb in embeddings2_1])  # Move tensors to CPU
      embeddings2_2_array = np.stack([emb.cpu() for emb in embeddings2_2])
      img_embeddings_array = np.stack([emb.cpu() for emb in img_embeddings])

      # Normalize the length of arrays to either cut off or repeat to reach 40 elements
      target_length = 40
      arrays = [ids_array, img_embeddings_array, embeddings1_1_array, embeddings1_2_array, embeddings2_1_array, embeddings2_2_array]
      adjusted_arrays = []

      for index, arr in enumerate(arrays):
          if len(arr) > target_length:
              adjusted_arrays.append(arr[:target_length])
          elif len(arr) < target_length:
              if index == 0:
                  # Repeatedly append elements until the array reaches the target length
                  extended_arr = np.tile(arr, (target_length // len(arr) + 1,))[:target_length]
                  #print(f"Len array: {arr.shape}. Len extended_arr: {len(extended_arr)}. Len adjust: {(target_length // len(arr) + 1,)}")
                  adjusted_arrays.append(extended_arr)
              else:
                  # Repeatedly append elements until the array reaches the target length
                  extended_arr = np.tile(arr, (target_length // len(arr) + 1, 1))[:target_length]
                  #print(f"Len array: {arr.shape}. Len extended_arr: {len(extended_arr)}. Len adjust: {(target_length // len(arr) + 1,)}")
                  adjusted_arrays.append(extended_arr)
          else:
              adjusted_arrays.append(arr)

      # Decompose the list of adjusted arrays back to individual arrays
      ids_array2, img_embeddings_array2, embeddings1_1_array2, embeddings1_2_array2, embeddings2_1_array2, embeddings2_2_array2 = adjusted_arrays

      # Save the arrays into an .npz file
      np.savez(npz_file_path, ids=ids_array, img_embeddings = img_embeddings_array, embeddings1_1=embeddings1_1_array, embeddings1_2=embeddings1_2_array, embeddings2_1=embeddings2_1_array, embeddings2_2=embeddings2_2_array)
      np.savez(npz_file_path2, ids=ids_array2, img_embeddings = img_embeddings_array2, embeddings1_1=embeddings1_1_array2, embeddings1_2=embeddings1_2_array2, embeddings2_1=embeddings2_1_array2, embeddings2_2=embeddings2_2_array2)
    else:
      print(f"No valid .txt files or embeddings generated for folder: {folder_name}")

  1%|          | 32/3997 [00:00<00:26, 147.00it/s]

9ad2dcb4-4ae8-4b99-a1ed-cbeb813b8c5b
9ad2dcb4-4ae8-4b99-a1ed-cbeb813b8c5b already done
fdf9fd10-8a2a-4cb4-bed0-3c476c976ce0
fdf9fd10-8a2a-4cb4-bed0-3c476c976ce0 already done
bbfee68d-1544-47cb-be5a-98e879bbe3ad
bbfee68d-1544-47cb-be5a-98e879bbe3ad already done
96776df9-9167-4ff8-af7b-4d9036c431c8
96776df9-9167-4ff8-af7b-4d9036c431c8 already done
e9df7231-410f-4f4e-a586-37063bc940e1
e9df7231-410f-4f4e-a586-37063bc940e1 already done
f0eed6bf-9b1b-4454-8595-ef9d990883d8
f0eed6bf-9b1b-4454-8595-ef9d990883d8 already done
f97c4058-cc2b-43da-97e5-96bef7e55f40
f97c4058-cc2b-43da-97e5-96bef7e55f40 already done
128fba8e-a03a-444a-8a79-c99b31cda0ea
128fba8e-a03a-444a-8a79-c99b31cda0ea already done
b2f60e78-7dca-4cb3-bc55-c62580be8010
b2f60e78-7dca-4cb3-bc55-c62580be8010 already done
797c5264-2edc-4718-b3fc-ee30b343a0fd
797c5264-2edc-4718-b3fc-ee30b343a0fd already done
96f24cce-09ca-41af-9bfb-73290fb5300c
96f24cce-09ca-41af-9bfb-73290fb5300c already done
cccf4591-da54-4f3b-8522-5664bb1b2a42
cccf45

  2%|▏         | 91/3997 [00:00<00:16, 238.40it/s]

a9ae7a19-be18-4737-8eb1-af560eb177de already done
0da4dac0-c294-4268-b0cf-00cb2db51e45
0da4dac0-c294-4268-b0cf-00cb2db51e45 already done
59690ef7-7dd5-4644-9081-338a1863b883
59690ef7-7dd5-4644-9081-338a1863b883 already done
58182588-b18f-4bc4-a7fd-3805c408b04b
58182588-b18f-4bc4-a7fd-3805c408b04b already done
3ac87d22-58ae-49e6-a4e8-59f1f9207686
3ac87d22-58ae-49e6-a4e8-59f1f9207686 already done
89f69402-5984-4546-b5b1-fefc5ba1f7e2
89f69402-5984-4546-b5b1-fefc5ba1f7e2 already done
1f73ba2e-c116-4c50-b1ff-a99bded21722
1f73ba2e-c116-4c50-b1ff-a99bded21722 already done
5aed651f-3103-4fee-9ec1-f21efbda4cb3
5aed651f-3103-4fee-9ec1-f21efbda4cb3 already done
cc25bced-af79-4a52-b03d-cf71d24a7adf
cc25bced-af79-4a52-b03d-cf71d24a7adf already done
66120159-5fe7-4cdb-8c9a-7ac68e393c6f
66120159-5fe7-4cdb-8c9a-7ac68e393c6f already done
d330ab6a-d70d-4e0e-8feb-a2e10cb1e7c5
d330ab6a-d70d-4e0e-8feb-a2e10cb1e7c5 already done
806139a2-556b-45d8-bc5e-93e3d898e4ad
806139a2-556b-45d8-bc5e-93e3d898e4ad alread

  4%|▍         | 153/3997 [00:00<00:14, 273.70it/s]

...

ada7a83c-e3e1-40f1-93f9-3e73dbc9298a already done
61568439-b790-4e96-9b4f-64503ce0ab12
61568439-b790-4e96-9b4f-64503ce0ab12 already done
7fcbcd27-33bc-4c40-bb45-5e3927004fac
7fcbcd27-33bc-4c40-bb45-5e3927004fac already done
1ed00158-c6a3-45cd-819c-2f91997fc480
1ed00158-c6a3-45cd-819c-2f91997fc480 already done
9724aa19-fdf9-49d0-9f9d-944e462e2a5c
9724aa19-fdf9-49d0-9f9d-944e462e2a5c already done
9ffa627f-32c2-4cb1-bfad-00cf348abc70
9ffa627f-32c2-4cb1-bfad-00cf348abc70 already done
535afeda-2538-435d-9dd1-5e10be586774
535afeda-2538-435d-9dd1-5e10be586774 already done
10876f5a-9ac7-49c7-a2b6-46a3f1ac3523
10876f5a-9ac7-49c7-a2b6-46a3f1ac3523 already done
d2db1078-c170-4e22-b261-14f3ad60a690
d2db1078-c170-4e22-b261-14f3ad60a690 already done
76c9a186-75bd-436a-85c0-823e3efddb7f
76c9a186-75bd-436a-85c0-823e3efddb7f already done
a92e181b-9d97-4442-8af2-a8967284c974
a92e181b-9d97-4442-8af2-a8967284c974 already done
d0630a08-3b40-4cb4-9f48-7d525262c1f6
d0630a08-3b40-4cb4-9f48-7d525262c1f6 alread

100%|█████████▉| 3984/3997 [00:21<00:00, 264.68it/s]

b98c9e55-6964-4c62-9804-bd325bd15d70 already done
0e763b87-f3cd-4a63-986f-bebc4189f1f0
0e763b87-f3cd-4a63-986f-bebc4189f1f0 already done
a7409219-a681-4072-adb2-5285106ce6f2
a7409219-a681-4072-adb2-5285106ce6f2 already done
28537bf2-22c5-4840-91fc-fcb98e8c7379
28537bf2-22c5-4840-91fc-fcb98e8c7379 already done
34e10b51-b5c6-4bc1-b70e-f05f141eda1e
34e10b51-b5c6-4bc1-b70e-f05f141eda1e already done
c433b570-1cb8-472a-bed7-4e24f94ea665
c433b570-1cb8-472a-bed7-4e24f94ea665 already done
469d6414-1f06-43de-80d5-17762d4a356a
469d6414-1f06-43de-80d5-17762d4a356a already done
cf33388f-96db-4f15-83eb-f5faacbf8f99
cf33388f-96db-4f15-83eb-f5faacbf8f99 already done
6eea322e-ab09-451f-818b-a1348ce14b91
6eea322e-ab09-451f-818b-a1348ce14b91 already done
5e35a109-b46f-47a3-9476-757f75cffc65
5e35a109-b46f-47a3-9476-757f75cffc65 already done
067102ea-9519-4622-9077-57ca4164cfbb
067102ea-9519-4622-9077-57ca4164cfbb already done
50a45c3d-b54b-4fd5-bb83-5046e4ff6813
50a45c3d-b54b-4fd5-bb83-5046e4ff6813 alread

100%|██████████| 3997/3997 [00:21<00:00, 185.48it/s]

4b6ce115-b3d7-493b-b14c-6f1c53add713
4b6ce115-b3d7-493b-b14c-6f1c53add713 already done


In [ ]:
artist = os.path.join(artists_dir, '3f23c754-8049-44bd-92ca-44a4bacb4380')
ids, img_embeddings, embeddings1_1, embeddings1_2, embeddings2_1, embeddings2_2 = process_artist(artist)
ids

PROCESS FUNCTION
Everything ok


[]

In [ ]:
# CHECK OF SOME MISSING .npz FILES
import os
from tqdm import tqdm

# Define the paths to the directories
directory1 = os.path.join(root_dir, 'instagram')
directory2 = os.path.join(root_dir, 'Embeddings_40')

# Get a list of folders in directory1
folders = [folder for folder in os.listdir(directory1)
           if os.path.isdir(os.path.join(directory1, folder))]

# Get a list of .npz files in directory2
npz_files = [file for file in os.listdir(directory2)
             if file.endswith('.npz')]

# Create a set of the base names of the npz files without the extension
npz_file_names = set(os.path.splitext(file)[0] for file in npz_files)

# Check each folder against the npz files
for folder in tqdm(folders):
    if not (folder in npz_file_names):
        print(f"No match found: Folder '{folder}' does not have a corresponding .npz file in directory2.")

100%|██████████| 3997/3997 [00:00<00:00, 543494.56it/s]

No match found: Folder '3f23c754-8049-44bd-92ca-44a4bacb4380' does not have a corresponding .npz file in directory2.
No match found: Folder '08870a97-6af5-4db8-97fa-859bb9f2e0c2' does not have a corresponding .npz file in directory2.
No match found: Folder 'cdcd53c9-f37c-48cc-b7e3-dfe34be22e92' does not have a corresponding .npz file in directory2.
No match found: Folder '1550f952-c91b-40d7-9b4d-d26a259ee932' does not have a corresponding .npz file in directory2.


In [ ]:
#CHECK OF THE SHAPES OF EVERY .npz FILE

import os
import numpy as np

# Specify the directory containing the .npz files
emb_dir = os.path.join(root_dir, 'Embeddings_40')

# Function to check the array shapes in the npz file
def check_shapes(file_path):
    with np.load(file_path) as data:
        ids = data['ids']
        img_embb = data['img_embeddings']
        emb1_1 = data['embeddings1_1']
        emb1_2 = data['embeddings1_2']
        emb2_1 = data['embeddings2_1']
        emb2_2 = data['embeddings2_2']

        # Check the lengths of all arrays and the specific shape for emb2_2
        if (len(ids) == 40 and len(img_embb) == 40 and
            len(emb1_1) == 40 and len(emb1_2) == 40 and
            len(emb2_1) == 40 and len(emb2_2) == 40 and
            emb2_2.shape == (40, 768)):
            return True
        else:
            return False

# Check each .npz file in the directory
for filename in os.listdir(emb_dir):
    if filename.endswith('.npz'):
        file_path = os.path.join(emb_dir, filename)
        if not check_shapes(file_path):
            print(f"{filename}: Shapes are NOT correct.")
